In [1]:
from rtmag.paper.load import load_input_label, MyModel

In [2]:
mm = MyModel("/home/mgjeon/space/workspace/base_important/uno_pi_cc_hnorm_unit_aug/model_75.pt")

Model loaded from epoch 75


In [3]:
input_files, label_files = load_input_label("/mnt/f/isee_dataset/11158")
len(input_files), len(label_files)

(66, 66)

In [4]:
idx = 13
input_files[idx]

PosixPath('/mnt/f/isee_dataset/11158/input/input_11158_20110214_000000.npz')

In [5]:
b = mm.get_pred(input_files[idx])

In [6]:
B = mm.get_label(label_files[idx])
Bp = mm.get_pot(label_files[idx])
dx, dy, dz, dV = mm.get_dV(input_files[idx])

In [7]:
import numpy as np

In [8]:
Btest = "/home/mgjeon/space/workspace/ar_nlfff_piml/notebook_paper/test_model/Wiegelmann/Bout_11158_20110214_000000.dat"

f = open(Btest, "r+")

#________________________________________________________________________
nx = int(512)
ny = int(256)
nz = int(256)

array_sz = (nz-1,ny-1,nx-1)

Bx = np.zeros(array_sz)
By = np.zeros(array_sz)
Bz = np.zeros(array_sz)

#_______________________________________________________________________

for iz in range (0, nz-1):
    for iy in range(0, ny-1):
        for ix in range(0, nx-1):
            for ivec in range(0,3):
                value = f.readline()
                
                try:
                    Bin = float(value)
                    if ivec == 0:
                        Bx[iz, iy, ix] = Bin
                    elif ivec == 1:
                        By[iz, iy, ix] = Bin
                    else:
                        Bz[iz, iy, ix] = Bin
            
                except:
                    Bin = 0
                    if ivec == 0:
                        Bx[iz, iy, ix] = Bin
                    elif ivec == 1:
                        By[iz, iy, ix] = Bin
                    else:
                        Bz[iz, iy, ix] = Bin

outBx = Bx.transpose(2,1,0)
outBy = By.transpose(2,1,0)
outBz = Bz.transpose(2,1,0)

In [9]:
b_W = np.stack((outBx, outBy, outBz), axis=-1)
b_W = np.pad(b_W, ((0, 1), (0, 1), (0, 1), (0, 0)), mode='constant')
b_W.shape

(512, 256, 256, 3)

In [10]:
import rtmag.paper.metric as m

In [11]:
# PINO
res = m.evaluate(b, B)

C_vec     : 0.96
C_cs      : 0.96
E_n'      : 0.69
E_m'      : 0.60
eps       : 0.94


In [12]:
# Wiegelmann
res = m.evaluate(b_W, B)

C_vec     : 0.95
C_cs      : 0.77
E_n'      : 0.59
E_m'      : 0.36
eps       : 0.83


In [13]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

In [ ]:
left_title = "Ground Truth"
right_title = "Prediction"
title_y = 1.10

title_fontsize = 25
bar_title_fontsize = 15
bar_ticks_fontsize = 10

label_fontsize = 12
tick_fontsize = 10